# Making A Witch

## Preprocessing 

Taking the final texts not in txt. form and transforming them

In [1]:
!pip install pdfplumber
!pip install matplotlib
!pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.2 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 31.8 MB/s eta 0:00:00


In [3]:
import pdfplumber
import matplotlib.pyplot as plt

# Define the y-axis range of interest
y_min = 70  # the y-coordinate to get rid of the header on all the text
y_max = 650 # and the same to get rid of the footer

# Define words to exclude
exclude_words = {"Author's note:", "Author", "Author's note"} #words that crop up at the beginning of every chapter but are too low down to cut it out like we do with the header and footer

# Open the PDF
with pdfplumber.open("16th-century-defence-of-witchcraft.pdf") as pdf:
    all_filtered_text = ""
    
    for page_number, page in enumerate(pdf.pages):
        # Extract all characters on the page
        characters = page.chars
        
        # (Optional) Visualize bounding boxes
        fig, ax = plt.subplots(figsize=(10, 15))
        ax.set_xlim(0, page.width)
        ax.set_ylim(0, page.height)
        ax.invert_yaxis()  # Match PDF coordinate system
        
        for char in characters:
            x0, top, x1, bottom = char["x0"], char["top"], char["x1"], char["bottom"]
            if y_min <= top <= y_max:  # Only include characters within the y-axis range
                ax.add_patch(plt.Rectangle((x0, top), x1 - x0, bottom - top, edgecolor="red", facecolor="none"))
        
        # Show the visualization for debugging (optional)
        plt.show()  #we know it looks good now so dont need to run this part
        
        # Extract text only from characters within the specified y-axis range
        filtered_text = ''.join(
            char["text"] for char in characters if y_min <= char["top"] <= y_max
        )
        
        # Split the filtered text into words, excluding unwanted words
        words = filtered_text.split()
        filtered_words = [word for word in words if word not in exclude_words]
        
        # Join the remaining words back into a string
        all_filtered_text += ' '.join(filtered_words) + "\n"
    
    # Save the filtered text to a .txt file
    with open("defense_of_witchcraft_16th_century_raw.txt", "w", encoding="utf-8") as output_file:
        output_file.write(all_filtered_text)
 #this was also edited to remove notes from the editor
        #SUCCESS, time for the next one


Text has been successfully extracted and combined into 'combined_output.txt'.


In [4]:
import fitz  # PyMuPDF, using a slightly different method, as for some reason when run on this pdf, it joins words together that shouldnt be, an error that did not occur with the above method

# Define the y-axis range of interest (to remove header and footer), these coordinates were found using the previous code, but taken out for conciseness
y_min = 20  # the y-coordinate to get rid of the header
y_max = 810 # and the same to get rid of the footer

# Define words to exclude (these often crop up at the beginning of every chapter or page)
exclude_words = {"Next", "Neo-Paganism", "Sacred-Texts", "Index", "Previous"} 

#creating functions to be able to use both methods, so word exclusion and coordinates can still be used

# Step 1: Using fitz (PyMuPDF) for additional text extraction from the PDF
def extract_text_using_fitz(pdf_path):
    document = fitz.open(pdf_path)
    all_filtered_text_fitz = ""
    
    for page_number in range(document.page_count):
        page = document.load_page(page_number)
        # Extract text from the page
        text = page.get_text("text")
        # Optionally, filter out unwanted words
        filtered_text = ' '.join([word for word in text.split() if word not in exclude_words])
        all_filtered_text_fitz += filtered_text + "\n"
    
    return all_filtered_text_fitz

# Step 2: Using pdfplumber for filtering by y-axis and word exclusion
def extract_text_using_pdfplumber(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_filtered_text_plumber = ""
        
        for page_number, page in enumerate(pdf.pages):
            # Extract all characters on the page
            characters = page.chars

            # Extract text only from characters within the specified y-axis range
            filtered_text = ''.join(
                char["text"] for char in characters if y_min <= char["top"] <= y_max
            )
            
            # Split the filtered text into words
            words = filtered_text.split()
            
            # Filter out unwanted words
            filtered_words = [word for word in words if word not in exclude_words]
            
            # Join the remaining words back into a string
            all_filtered_text_plumber += ' '.join(filtered_words) + "\n"
        
        return all_filtered_text_plumber

# Combine results from both methods: fitz and pdfplumber
def combine_text_from_both_methods(pdf_path):
    # Extract text using both methods
    text_fitz = extract_text_using_fitz(pdf_path)
    text_plumber = extract_text_using_pdfplumber(pdf_path)
    
    # Combine the extracted text from both methods
    combined_text = text_fitz + "\n\n" + "===" * 20 + "\n\n" + text_plumber
    
    # Save the combined text to a .txt file
    with open("combined_output.txt", "w", encoding="utf-8") as output_file:
        output_file.write(combined_text) #this name is later changed to malleus_maleficarum-1487.txt, time stamps and https: etc etc, were still included so these were manually changed in the txt file
    #the name change occured to mark that it was the finished product
    print("Text has been successfully extracted and combined into 'combined_output.txt'.")

combine_text_from_both_methods("Malleus_Maleficarum_combined.pdf")


Text has been successfully extracted and combined into 'combined_output.txt'.


There are a lot of errors in the text, but no amount of attempts to streamline correcting them are working, so i will manually read through the texts and ensure they are correct

All the other texts are already in txt format, via copy and pasting from the original pdf as they are all rather small, so was simpler to do it that way, so is only necessary for these two larger texts